In [1]:
# import all packages
 
import numpy as np

In [2]:
data = np.loadtxt('DMDdata200.csv', delimiter=',')

In [3]:
x1 = data[0:,1:-1]
x2 = data[0:,2:]

In [4]:
data_shape = (x1.shape[1], x1.shape[0])
percent_Trunc = 0.90
Frate = 1/2.5e-05
dt = 2.5e-05

In [5]:
# Compression matrix generation

C_gaussian = np.random.normal(0, 1, size=data_shape)

In [6]:
# compress data

y1 = np.dot(C_gaussian, x1)
y2 = np.dot(C_gaussian, x2)

In [7]:
# cDMD
start = 0
end = 0
import time
start = time.time()

[Uc, Sc, Vc] = np.linalg.svd(y1, full_matrices=False, compute_uv=True, hermitian=False)

menergyc = 0.0
for i in range(0, np.size(Sc)):
    menergyc += Sc[i]/np.sum(Sc)
    if menergyc > percent_Trunc:
        break
    rc = i
    
menergyc, rc

Atildec = Uc[:, :rc].T @ y2 @ Vc[:, :rc] * np.reciprocal(Sc[: rc])
[eivValc,eivVecc] = np.linalg.eig(Atildec)

phic = x2 @ Vc[:rc, :].T @ np.diag(np.reciprocal(Sc[: rc])) @ eivVecc
bc = np.linalg.lstsq(phic, x1[:, 0])

num_modesc = phic.shape[1]
omegac = np.log(eivValc) / dt

end = time.time()
print(end - start)

5.423640012741089


C:\Users\sumuk\AppData\Local\Temp\ipykernel_16956\1308613788.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  bc = np.linalg.lstsq(phic, x1[:, 0])
C:\Users\sumuk\AppData\Local\Temp\ipykernel_16956\1308613788.py:25: RuntimeWarning: invalid value encountered in log
  omegac = np.log(eivValc) / dt


In [8]:
# DMD

start = 0
end = 0

import time
start = time.time()

[U, S, V] = np.linalg.svd(x1, full_matrices=False, compute_uv=True, hermitian=False)

menergy = 0.0
for i in range(0, np.size(S)):
    menergy += S[i]/np.sum(S)
    if menergy > percent_Trunc:
        break
    r = i
    
menergy, r

Atilde = U[:, :r].T @ x2 @ V[:, :r] * np.reciprocal(S[: r])
[eivVal,eivVec] = np.linalg.eig(Atilde)

phi = x2 @ V[:r, :].T @ np.diag(np.reciprocal(S[: r])) @ eivVec # Compute full-state modes of X
b = np.linalg.lstsq(phi, x1[:, 0])

num_modes = phi.shape[1]
omega = np.log(eivVal) / dt

end = time.time()
print(end - start)

41.52670383453369


C:\Users\sumuk\AppData\Local\Temp\ipykernel_16956\1158266998.py:24: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  b = np.linalg.lstsq(phi, x1[:, 0])


In [11]:
## Reconstruction of the data

def recon(phi, b, omega, num_steps, r):
    time_dynamics = np.zeros((r, num_steps), dtype=complex)
    for i in range(num_steps):
        time_dynamics[:, i] = b[0] * np.exp(omega * i)
    return phi @ time_dynamics

In [12]:
reconstructed_fieldc = recon(phic, bc, omegac, 2000, rc)
reconstructed_field = recon(phi, b, omega, 2000, r)